![elpretty](malumabby.jpg)
# 🐶 Sistema Experto con Incertidumbre para Identificar Razas de Perros

En este cuaderno construyo un **sistema experto** capaz de **sugerir la raza más probable** de un perro a partir de rasgos observables (tamaño, tipo de pelo, orejas, hocico, cola, color, etc.), considerando que las descripciones reales suelen ser **imprecisas** o **parciales**.

---

## 🎯 Objetivo
Dado un conjunto de **hechos con grado de certeza** (por ejemplo: *“orejas paradas” = 0.8*), el sistema infiere una o varias **razas candidatas** y las ordena por **Factor de Certeza (CF)**.

---

## 🧠 Componentes del sistema
- **Base de conocimiento:** reglas tipo **SI (rasgos) ENTONCES (raza)** con un **CF** que indica qué tan representativa es la regla.
- **Memoria de trabajo:** hechos del caso actual (rasgos ingresados) con su incertidumbre.
- **Motor de inferencia:** encadenamiento hacia adelante que:
  - calcula la certeza de los antecedentes (AND → `min`)
  - propaga certeza a conclusiones (CF_antecedentes × CF_regla)
  - combina evidencias múltiples (función `combinar_cf`)
- **Módulo de explicación:** registra qué reglas se activaron y cuánto aportó cada una al resultado.

---

## 🔎 Qué produce el sistema
- Un **ranking** de razas con su CF final (más alto = más probable).
- Una **traza** explicable: “por qué” se recomendó esa raza.

---

## 🧪 Flujo del cuaderno
1. Definir rasgos (hechos) y cómo se ingresan con incertidumbre.
2. Definir reglas y sus factores de certeza.
3. Ejecutar el motor de inferencia.
4. Mostrar ranking + explicación.

> Nota: Los **CF no son probabilidades**, son una medida de **confianza** basada en evidencia y reglas.

In [1]:
from dataclasses import dataclass
from typing import List, Dict, Tuple

In [2]:
@dataclass
class Regla:
    si: List[str]          # lista de hechos requeridos
    entonces: str          # raza
    fc: float              # fiabilidad de la regla ([-1, 1])
    id: str = ""           # para explicación


In [3]:
def combinar_cf(cf1: float, cf2: float) -> float:
    # Ambos a favor
    if cf1 >= 0 and cf2 >= 0:
        return cf1 + cf2 * (1 - cf1)
    # Ambos en contra
    if cf1 <= 0 and cf2 <= 0:
        return cf1 + cf2 * (1 + cf1)
    # Conflicto
    return (cf1 + cf2) / (1 - min(abs(cf1), abs(cf2)))

## ⚙️ Motor de inferencia (Forward Chaining / Encadenamiento hacia adelante)

Este sistema experto utiliza **encadenamiento hacia adelante (forward chaining)**: parte de los **hechos observados** (rasgos del perro con su grado de certeza) y, a partir de ellos, va aplicando reglas de la base de conocimiento para **inferir conclusiones** (razas candidatas) con un **Factor de Certeza (CF)**.

### 🧩 ¿Cómo funciona?
Para cada regla del tipo:

**SI** (rasgos) **ENTONCES** (raza) con un `CF_regla`

1. **Verificación de condiciones**  
   La regla solo se evalúa si **todas** sus condiciones están presentes en la memoria de trabajo (`hechos`).

2. **Certeza de los antecedentes (AND → `min`)**  
   Como la regla representa una conjunción ("A y B y C"), la certeza de los antecedentes se calcula como:
   \[
   CF_{ant} = \min(CF(A), CF(B), CF(C))
   \]
   (la condición menos segura limita la certeza total).

3. **Propagación hacia la conclusión**  
   El aporte de esa regla a la raza se calcula como:
   \[
   CF_{aporte} = CF_{ant} \times CF_{regla}
   \]
   Si `CF_regla` es negativo, el aporte se interpreta como **evidencia en contra** de esa raza.

4. **Combinación de evidencias**  
   Si varias reglas concluyen la misma raza, sus aportes se fusionan con una función de combinación (`combinar_cf`) que:
   - refuerza cuando las evidencias coinciden,
   - y atenúa/normaliza cuando hay conflicto (una a favor y otra en contra).

5. **Ranking final**  
   Al final, el sistema genera un **ranking** de razas ordenadas por su `CF` final (más alto = más probable), y además guarda un **log explicable** con las reglas que contribuyeron a cada conclusión.

> Nota: Este motor es *forward* porque **no parte de una hipótesis** (“¿será Husky?”) para buscar evidencia, sino que **parte de evidencia** (rasgos) para llegar a conclusiones.

In [4]:
def inferir(hechos: Dict[str, float], reglas: List[Regla], epsilon: float = 1e-3):
    conclusiones: Dict[str, float] = {}  # raza -> CF final
    logs: Dict[str, List[dict]] = {}   # raza -> lista de aportes

    hubo_cambio = True
    while hubo_cambio:
        hubo_cambio = False

        for r in reglas:
            if all(h in hechos for h in r.si):
                fc_ant = min(hechos[h] for h in r.si)
                fc_aporte = fc_ant * abs(r.fc)
                if r.fc < 0:
                    fc_aporte *= -1

                prev = conclusiones.get(r.entonces, 0.0)
                nuevo = combinar_cf(prev, fc_aporte)

                if abs(nuevo - prev) > epsilon:
                    conclusiones[r.entonces] = nuevo
                    hubo_cambio = True

                logs.setdefault(r.entonces, []).append({
                    "regla": r.id,
                    "si": r.si,
                    "fc_antecedentes": round(fc_ant, 4),
                    "fc_regla": r.fc,
                    "fc_aporte": round(fc_aporte, 4),
                    "cf_prev": round(prev, 4),
                    "cf_nuevo": round(nuevo, 4),
                })

    ranking = sorted(conclusiones.items(), key=lambda x: x[1], reverse=True)
    return ranking, logs

In [5]:
# -----------------------------
# EJEMPLO DE BASE DE CONOCIMIENTO (puedes ampliar)
# -----------------------------
REGLAS = [
    Regla(["tamano_grande", "pelo_medio", "orejas_paradas", "mascara_cara", "cola_poblada"], "Pastor Aleman", 0.85, "R1"),
    Regla(["tamano_grande", "pelo_largo", "ojos_azules", "orejas_paradas", "cola_poblada"], "Husky Siberiano", 0.90, "R2"),
    Regla(["tamano_mediano", "pelo_corto", "orejas_caidas", "tricolor"], "Beagle", 0.88, "R3"),
    Regla(["tamano_grande", "dorado", "pelo_medio", "orejas_caidas"], "Golden Retriever", 0.86, "R4"),
    Regla(["tamano_grande", "pelo_corto", "robusto", "negro_y_fuego"], "Rottweiler", 0.80, "R5"),
    Regla(["tamano_pequeno", "pelo_largo", "cuerpo_alargado", "orejas_caidas"], "Dachshund", 0.87, "R6"),
    Regla(["tamano_pequeno", "hocico_corto", "orejas_paradas", "cuerpo_robusto"], "Bulldog Frances", 0.90, "R7"),
    Regla(["tamano_pequeno", "pelo_rizado", "orejas_caidas"], "Poodle", 0.85, "R8"),
    Regla(["tamano_grande", "pelo_corto", "dorado", "orejas_caidas"], "Labrador Retriever", 0.75, "R9"),
    # Regla "anti-evidencia" (si esto, entonces NO es X) usando FC negativo:
    Regla(["pelo_rizado"], "Labrador Retriever", -0.60, "R10"),
]


In [6]:
# -----------------------------
# EJEMPLO DE HECHOS (entrada del usuario con incertidumbre)
# -----------------------------
hechos = {
    "tamano_grande": 0.9,
    "pelo_largo": 0.8,
    "ojos_azules": 0.7,
    "orejas_paradas": 0.9,
    "cola_poblada": 0.8
}


In [10]:
ranking, logs = inferir(hechos, REGLAS)

print("Top razas:")
for raza, cf in ranking[:5]:
    print(f"- {raza}: {cf:.3f}")

print("\nExplicación de la mejor:")
mejor = ranking[0][0] if ranking else None
if mejor:
    for t in logs[mejor]:
        print(t)

Top razas:
- Husky Siberiano: 0.999

Explicación de la mejor:
{'regla': 'R2', 'si': ['tamano_grande', 'pelo_largo', 'ojos_azules', 'orejas_paradas', 'cola_poblada'], 'fc_antecedentes': 0.7, 'fc_regla': 0.9, 'fc_aporte': 0.63, 'cf_prev': 0.0, 'cf_nuevo': 0.63}
{'regla': 'R2', 'si': ['tamano_grande', 'pelo_largo', 'ojos_azules', 'orejas_paradas', 'cola_poblada'], 'fc_antecedentes': 0.7, 'fc_regla': 0.9, 'fc_aporte': 0.63, 'cf_prev': 0.63, 'cf_nuevo': 0.8631}
{'regla': 'R2', 'si': ['tamano_grande', 'pelo_largo', 'ojos_azules', 'orejas_paradas', 'cola_poblada'], 'fc_antecedentes': 0.7, 'fc_regla': 0.9, 'fc_aporte': 0.63, 'cf_prev': 0.8631, 'cf_nuevo': 0.9493}
{'regla': 'R2', 'si': ['tamano_grande', 'pelo_largo', 'ojos_azules', 'orejas_paradas', 'cola_poblada'], 'fc_antecedentes': 0.7, 'fc_regla': 0.9, 'fc_aporte': 0.63, 'cf_prev': 0.9493, 'cf_nuevo': 0.9813}
{'regla': 'R2', 'si': ['tamano_grande', 'pelo_largo', 'ojos_azules', 'orejas_paradas', 'cola_poblada'], 'fc_antecedentes': 0.7, 'fc_r

## ✅ Tests (Casos de prueba)

En esta sección se ejecutan **casos de prueba** para validar que el sistema experto:
- identifica correctamente razas en escenarios claros,
- maneja incertidumbre (CF parciales),
- y responde de forma coherente ante conflictos o información insuficiente.

### Todo cambio debe ser testeado 

In [8]:
# =========================
# TESTS (casos de prueba)
# =========================
# Nota: estos tests asumen inferencia de 1 pasada (sin “acumular” la misma regla en un while).
# Si tu inferir tiene while, los CF tienden a saturarse y el orden puede variar.

def inferir_1pasada(hechos, reglas):
    conclusiones = {}
    logs = {}

    for r in reglas:
        if all(h in hechos for h in r.si):
            fc_ant = min(hechos[h] for h in r.si)
            fc_aporte = fc_ant * abs(r.fc)
            if r.fc < 0:
                fc_aporte *= -1

            prev = conclusiones.get(r.entonces, 0.0)
            nuevo = combinar_cf(prev, fc_aporte)

            conclusiones[r.entonces] = nuevo
            logs.setdefault(r.entonces, []).append({
                "regla": r.id,
                "si": r.si,
                "fc_antecedentes": round(fc_ant, 4),
                "fc_regla": r.fc,
                "fc_aporte": round(fc_aporte, 4),
                "cf_prev": round(prev, 4),
                "cf_nuevo": round(nuevo, 4),
            })

    ranking = sorted(conclusiones.items(), key=lambda x: x[1], reverse=True)
    return ranking, logs


TESTS = [
    {
        "nombre": "1) Husky claro",
        "hechos": {"tamano_grande":0.9,"pelo_largo":0.85,"ojos_azules":0.8,"orejas_paradas":0.9,"cola_poblada":0.8},
        "top_esperado": "Husky Siberiano",
    },
    {
        "nombre": "2) Pastor Alemán claro",
        "hechos": {"tamano_grande":0.9,"pelo_medio":0.8,"orejas_paradas":0.9,"mascara_cara":0.85,"cola_poblada":0.8},
        "top_esperado": "Pastor Aleman",
    },
    {
        "nombre": "3) Beagle claro",
        "hechos": {"tamano_mediano":0.9,"pelo_corto":0.9,"orejas_caidas":0.95,"tricolor":0.8},
        "top_esperado": "Beagle",
    },
    {
        "nombre": "4) Golden claro",
        "hechos": {"tamano_grande":0.9,"dorado":0.95,"pelo_medio":0.85,"orejas_caidas":0.8},
        "top_esperado": "Golden Retriever",
    },
    {
        "nombre": "5) Rottweiler claro",
        "hechos": {"tamano_grande":0.9,"pelo_corto":0.9,"robusto":0.85,"negro_y_fuego":0.8},
        "top_esperado": "Rottweiler",
    },
    {
        "nombre": "6) Dachshund claro",
        "hechos": {"tamano_pequeno":0.9,"pelo_largo":0.8,"cuerpo_alargado":0.9,"orejas_caidas":0.85},
        "top_esperado": "Dachshund",
    },
    {
        "nombre": "7) Bulldog Francés claro",
        "hechos": {"tamano_pequeno":0.95,"hocico_corto":0.9,"orejas_paradas":0.8,"cuerpo_robusto":0.85},
        "top_esperado": "Bulldog Frances",
    },
    {
        "nombre": "8) Poodle claro (y Labrador negativo por pelo_rizado)",
        "hechos": {"tamano_pequeno":0.8,"pelo_rizado":0.95,"orejas_caidas":0.9},
        "top_esperado": "Poodle",
    },
    {
        "nombre": "9) Labrador claro",
        "hechos": {"tamano_grande":0.9,"pelo_corto":0.85,"dorado":0.8,"orejas_caidas":0.85},
        "top_esperado": "Labrador Retriever",
    },
    {
        "nombre": "10) Conflicto Labrador (rasgos de labrador + pelo_rizado)",
        "hechos": {"tamano_grande":0.9,"pelo_corto":0.85,"dorado":0.8,"orejas_caidas":0.85,"pelo_rizado":0.7},
        "top_esperado": "Labrador Retriever",
    },
    {
        "nombre": "11) Ambiguo: Poodle vs Dachshund (+ Labrador negativo por rizado)",
        "hechos": {"tamano_pequeno":0.9,"pelo_largo":0.7,"cuerpo_alargado":0.8,"orejas_caidas":0.8,"pelo_rizado":0.7},
        "top_esperado": "Dachshund",  # con estas certezas, Dachshund queda un poquito arriba
    },
    {
        "nombre": "12) Insuficiente (no dispara reglas)",
        "hechos": {"tamano_grande":0.9,"orejas_paradas":0.9},
        "top_esperado": None,
    },
    {
        "nombre": "13) Husky parcial sin ojos azules (no dispara)",
        "hechos": {"tamano_grande":0.9,"pelo_largo":0.85,"orejas_paradas":0.9,"cola_poblada":0.8},
        "top_esperado": None,
    },
    {
        "nombre": "14) Golden vs Labrador (ambos disparan)",
        "hechos": {"tamano_grande":0.9,"dorado":0.9,"pelo_medio":0.7,"pelo_corto":0.7,"orejas_caidas":0.8},
        "top_esperado": "Golden Retriever",
    },
]


def run_tests():
    ok = 0
    for t in TESTS:
        ranking, logs = inferir_1pasada(t["hechos"], REGLAS)
        top = ranking[0][0] if ranking else None
        cf  = ranking[0][1] if ranking else None

        passed = (top == t["top_esperado"])
        status = "✅" if passed else "❌"
        print(f"{status} {t['nombre']} -> top={top} cf={None if cf is None else round(cf,3)}")

        if not passed:
            print("   esperado:", t["top_esperado"])
            print("   ranking:", [(r, round(c,3)) for r,c in ranking[:5]])
        else:
            ok += 1

    print(f"\nResultado: {ok}/{len(TESTS)} tests OK")


run_tests()

✅ 1) Husky claro -> top=Husky Siberiano cf=0.72
✅ 2) Pastor Alemán claro -> top=Pastor Aleman cf=0.68
✅ 3) Beagle claro -> top=Beagle cf=0.704
✅ 4) Golden claro -> top=Golden Retriever cf=0.688
✅ 5) Rottweiler claro -> top=Rottweiler cf=0.64
✅ 6) Dachshund claro -> top=Dachshund cf=0.696
✅ 7) Bulldog Francés claro -> top=Bulldog Frances cf=0.72
✅ 8) Poodle claro (y Labrador negativo por pelo_rizado) -> top=Poodle cf=0.68
✅ 9) Labrador claro -> top=Labrador Retriever cf=0.6
✅ 10) Conflicto Labrador (rasgos de labrador + pelo_rizado) -> top=Labrador Retriever cf=0.31
✅ 11) Ambiguo: Poodle vs Dachshund (+ Labrador negativo por rizado) -> top=Dachshund cf=0.609
✅ 12) Insuficiente (no dispara reglas) -> top=None cf=None
✅ 13) Husky parcial sin ojos azules (no dispara) -> top=None cf=None
✅ 14) Golden vs Labrador (ambos disparan) -> top=Golden Retriever cf=0.602

Resultado: 14/14 tests OK
